<a href="https://colab.research.google.com/github/qinfendebirddr/numpy-100/blob/master/%E2%80%9Cdemo_tabular_customer_churn_tensorflow2_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-05-08 15:05:00--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv’

customer_churn.csv  100%[===================>] 668.81K  --.-KB/s    in 0.06s   

2019-05-08 15:05:00 (10.6 MB/s) - ‘customer_churn.csv’ saved [684858/684858]



In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('customer_churn.csv')

In [0]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
df.columns

In [0]:
!pip install -q tensorflow==2.0.0-alpha0

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)

In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [0]:
train, valid = train_test_split(train, test_size=0.2, random_state=1)

In [0]:
print(len(train))
print(len(valid))
print(len(test))

In [0]:
feature_columns = []

In [0]:
numeric_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [0]:
for header in numeric_columns:
  feature_columns.append(feature_column.numeric_column(header))

In [0]:
categorical_columns = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

In [0]:
def get_one_hot_from_categorical(colname):
  categorical = feature_column.categorical_column_with_vocabulary_list(colname, train[colname].unique().tolist())
  return feature_column.indicator_column(categorical)

In [0]:
geography = get_one_hot_from_categorical('Geography'); geography

In [0]:
for col in categorical_columns:
  feature_columns.append(get_one_hot_from_categorical(col))

In [0]:
feature_columns

In [0]:
from tensorflow.keras import layers

In [0]:
feature_layer = layers.DenseFeatures(feature_columns); feature_layer

In [0]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

In [0]:
train_ds = df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

In [0]:
model = keras.Sequential([
  feature_layer,
  layers.Dense(200, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=5)

In [0]:
model.summary()

In [0]:
model.evaluate(test_ds)